In this assignment we are going to analyse scull measurement data of patients with malocclusion. Patients affected by Class III malocclusion (characterised by the protrusion of lower dental arch) suffer from a skeletal imbalance that is established early in life, and that becomes more pronounced during puberty and until skeletal maturation is complete. Predicting treatment success or failure early in a single Class III patient makes it easier to correct it, but it is difficult to do just from a small number of morphometric determinants is problematic. The reason for that is that Class III malocclusion is rarely a consequence of an abnormality in a single craniofacial component, so individual clinical and radiological measurements are likely to be less indicative than the interplay between the measurements themselves.

The data set we will use contains 143 patients with two sets of measurements at ages T1 and T2 (measured in years) for the following variables:
 - Treatment: untreated (0) or treated (1).
 - Growth: a binary variable with values Good or Bad, determined on the basis of CoGn-CoA.
 - ANB: angle between Down's points A and B (degrees).
 - IMPA: incisor-mandibular plane angle (degrees).
 - PPPM: palatal plane - mandibular plane angle (degrees).
 - CoA: total maxillary length from condilion to Down's point A (mm).
 - GoPg: length of mandibular body from gonion to pogonion (mm).
 - CoGo: length of mandibular ramus from condilion to pogonion (mm).
 
For simplicity, we transform the dataset by taking differences between time points T2 and T1, so all features now represent changes in measurements.
 
We would like to estimate the effect of the treatment on Growth and dANB by taking into account causal relationships between variables. Our knowledge of those causal relationships is represented on the following DAG:
![title](dag_final.jpg)

Treatment assignment, Growth and number of years between two measurements are likely to be affected by some unobserved confounders, as the graph shows.

Given the graph, select the variables to condition on, apply suitable adjustment method, and calculate your estimates of causal effects: Treatment on Growth, Treatment on dANB, both ATE and ATET.


Your analysis should contain:
 - Selection of covariates to adjust for (informed by the graph)
 - Application of the most suitable adjustment method
 - Estimates of the ATE and ATET

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.rcParams['figure.figsize'] = 10, 8


df = pd.read_csv('malocclusion.csv')
display(df)
df.describe()

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment
0,-3.2,-1.1,-4.2,1.0,4.0,3.7,5,0,0
1,-0.6,-0.5,3.8,2.6,-0.1,1.4,3,1,0
2,-1.6,-3.1,-6.0,4.3,4.2,7.1,5,0,0
3,-1.1,-2.1,-12.1,14.1,20.7,17.5,9,0,0
4,-1.1,0.0,-6.7,7.7,8.8,11.0,5,0,0
...,...,...,...,...,...,...,...,...,...
138,0.8,-2.1,-2.0,2.7,2.0,3.3,5,1,1
139,2.1,0.7,1.4,8.2,12.8,9.1,10,1,1
140,-0.2,-3.3,-2.7,6.8,3.4,10.9,4,1,1
141,1.5,-3.5,1.8,4.6,6.5,6.2,5,1,1


,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment
count,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000
mean,-0.227273,-1.374825,-0.785315,5.987413,7.730769,6.732867,4.706294,0.405594,0.461538
std,1.826225,2.715046,5.080894,4.469692,5.532417,4.595141,2.550427,0.492733,0.500271
min,-5.100000,-9.300000,-19.000000,-0.900000,-1.400000,-2.600000,1.000000,0.000000,0.000000
25%,-1.350000,-2.750000,-3.450000,1.800000,3.200000,3.050000,3.000000,0.000000,0.000000
50%,-0.300000,-1.400000,-0.400000,5.500000,6.200000,6.300000,4.000000,0.000000,0.000000
75%,0.950000,0.050000,2.100000,9.750000,12.750000,10.350000,6.000000,1.000000,1.000000
max,4.900000,6.500000,12.000000,20.000000,23.300000,17.500000,12.000000,1.000000,1.000000


**Taking into account our DAG, several things can be noticed:**
1. The dCOA indicator is a collider (in conjunction with Treatment and dANB)
2. The node "unobserved confounders" is a fork, so Treatment and dT are likely dependent, and in this regard we get the third point.
3. We have only one path to dANB and Growth (Treatment <- unobserved confounders -> dT -> Growth -> dANB)

### The effect of the treatment on Growth

Let's estimate (treatment - growth) for this we will adjust dT. So:

In [2]:
m = smf.ols('Growth ~ Treatment + dT', data=df)
fitted = m.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                 Growth   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     3.433
Date:                Wed, 26 Oct 2022   Prob (F-statistic):             0.0350
Time:                        14:19:15   Log-Likelihood:                -97.770
No. Observations:                 143   AIC:                             201.5
Df Residuals:                     140   BIC:                             210.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4623      0.086      5.382      0.0

#### Propensity score weighting
With classifier calibration.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

cls = LogisticRegression()
cls = CalibratedClassifierCV(cls)

X = df[['dT']]
y = df['Treatment']
cls.fit(X, y)
df['e'] = cls.predict_proba(X)[:,1].tolist()
display(df.head())

df['w'] = df['Treatment'] / df['e'] + (1 - df['Treatment']) / (1 - df['e'])
m = smf.wls('Growth ~ Treatment + dT', data=df, weights=df['w'])
fitted = m.fit()
print(fitted.summary())

,dANB,dPPPM,dIMPA,dCoA,dGoPg,dCoGo,dT,Growth,Treatment,e
0,-3.2,-1.1,-4.2,1.0,4.0,3.7,5,0,0,0.486891
1,-0.6,-0.5,3.8,2.6,-0.1,1.4,3,1,0,0.335406
2,-1.6,-3.1,-6.0,4.3,4.2,7.1,5,0,0,0.486891
3,-1.1,-2.1,-12.1,14.1,20.7,17.5,9,0,0,0.751201
4,-1.1,0.0,-6.7,7.7,8.8,11.0,5,0,0,0.486891


                            WLS Regression Results                            
Dep. Variable:                 Growth   R-squared:                       0.078
Model:                            WLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     5.881
Date:                Wed, 26 Oct 2022   Prob (F-statistic):            0.00353
Time:                        14:19:16   Log-Likelihood:                -99.753
No. Observations:                 143   AIC:                             205.5
Df Residuals:                     140   BIC:                             214.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4515      0.092      4.930      0.0

#### Doubly robust estimator

In [4]:
from sklearn.linear_model import LinearRegression

y0 = LinearRegression().fit(df[df.Treatment == 0][['dT']], df[df.Treatment == 0]['Growth']).predict(df[['dT']])
y1 = LinearRegression().fit(df[df.Treatment == 1][['dT']], df[df.Treatment == 1]['Growth']).predict(df[['dT']])

df['DR0'] = (1-df['Treatment']) * (df['Growth'] - y0)/(1-df['e']) + y0
df['DR1'] =    df['Treatment']  * (df['Growth'] - y1)/   df['e']  + y1
df['DR1'].mean() - df['DR0'].mean()

0.19901991595095325

### The effect of the treatment on dANB
Now let's evaluate (treatment - dANB), for this we will adjust Growth. So:

In [5]:
m = smf.ols('dANB ~ Treatment + Growth', data=df)
fitted = m.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                   dANB   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.398
Method:                 Least Squares   F-statistic:                     48.04
Date:                Wed, 26 Oct 2022   Prob (F-statistic):           1.31e-16
Time:                        14:19:16   Log-Likelihood:                -251.17
No. Observations:                 143   AIC:                             508.3
Df Residuals:                     140   BIC:                             517.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.5600      0.181     -8.609      0.0

#### Propensity score weighting

In [6]:
cls = LogisticRegression()
cls = CalibratedClassifierCV(cls)

X = df[['Growth']]
y = df['Treatment']
cls.fit(X, y)
df['e'] = cls.predict_proba(X)[:,1].tolist()
df['w'] = df['Treatment'] / df['e'] + (1 - df['Treatment']) / (1 - df['e'])
m = smf.wls('dANB ~ Treatment + Growth', data=df, weights=df['w'])
fitted = m.fit()
print(fitted.summary())

                            WLS Regression Results                            
Dep. Variable:                   dANB   R-squared:                       0.386
Model:                            WLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     44.06
Date:                Wed, 26 Oct 2022   Prob (F-statistic):           1.44e-15
Time:                        14:19:16   Log-Likelihood:                -253.68
No. Observations:                 143   AIC:                             513.4
Df Residuals:                     140   BIC:                             522.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.5572      0.205     -7.609      0.0

#### Doubly robust estimator

In [7]:
y0 = LinearRegression().fit(df[df.Treatment == 0][['dT', 'Growth']], df[df.Treatment == 0]['dANB']).predict(df[['dT', 'Growth']])
y1 = LinearRegression().fit(df[df.Treatment == 1][['dT', 'Growth']], df[df.Treatment == 1]['dANB']).predict(df[['dT', 'Growth']])

df['DR0'] = (1-df['Treatment']) * (df['dANB'] - y0)/(1-df['e']) + y0
df['DR1'] =    df['Treatment']  * (df['dANB'] - y1)/   df['e']  + y1
df['DR1'].mean() - df['DR0'].mean()

1.8796680991511714

#### Matching with Machalanobis distance

In [8]:
from causalinference import CausalModel
adjustment_set = ['Growth']

causal = CausalModel(
    Y=df['dANB'].values,
    D=df['Treatment'].values,
    X=df[adjustment_set].values
)
causal.est_via_matching(bias_adj=True)
print(causal.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      1.856      0.237      7.829      0.000      1.392      2.321
           ATC      1.860      0.240      7.761      0.000      1.390      2.330
           ATT      1.852      0.240      7.723      0.000      1.382      2.322



/home/matu/.local/lib/python3.8/site-packages/causalinference/estimators/matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef
